In [1]:
import pandas as pd
def securities_from_moex(kind=0, tiker=None):
  """Получение результатов торгов ценных бумаг на Московской бирже
  kind - может принимать следующие значения: 0 - акции, 1 - фьючерсы, 2 - валюты, 3 - индексы, 4 - облигации
  tiker - наименование тикера ценной бумаги, если не указан то будут выведены все бумаги за предыдущий день
  """
  if kind == 0:
    sUrl = 'http://iss.moex.com/iss/history/engines/stock/markets/shares/securities'
  elif kind == 1:
    sUrl = 'http://iss.moex.com/iss/history/engines/futures/markets/forts/securities'
  elif kind == 2:
    sUrl = 'http://iss.moex.com/iss/history/engines/currency/markets/selt/securities'
  elif kind == 3:
    sUrl = 'http://iss.moex.com/iss/history/engines/stock/markets/index/securities'
  else:
    sUrl = 'http://iss.moex.com/iss/history/engines/stock/markets/bonds/securities'
  if tiker is None:
    sUrl = sUrl + '.json'
  else:
    sUrl = sUrl + '/' + tiker + '.json'
  bNext = True
  lStart = 0
  while bNext:
    res = pd.read_json(sUrl + '?start=' + str(lStart))
    jhist = res['history']
    jdata = jhist['data']
    if lStart == 0:
      jcols = jhist['columns']
      df = pd.DataFrame(jdata, columns=jcols)
      if kind < 2:
        df.dropna(inplace=True)
      jcur = res['history.cursor']
      jcur = jcur['data']
      jcur = jcur[0]
      lTotal = jcur[1]
      lSize = jcur[2]
    else:
      res = pd.DataFrame(jdata, columns=jcols)
      if kind < 2:
        res.dropna(inplace=True)
      df = pd.concat([df, res], ignore_index=True)
    lStart = lStart + lSize
    if lStart < lTotal:
      bNext = True
    else:
      bNext = False
  return df
# Получение новых значений по фьючерсам MOEX
df1 = securities_from_moex(kind=1)